# Filter Reads: Select reads with high quality

According to the project, the high quality sequence are defined as **being each member of the pair has at least 50 consecutive bases of quality score 25+** 

In [1]:
import numpy as np
import itertools as it
import toolz as tz
import toolz.curried as c
import os

from fun_fastq import stream_fastq, write_fastq, is_high_quality
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

DIR_DAT_RAW    = "/home/jovyan/work/Data/SRR4841864/rawdata"
DIR_DAT_TRIM   = "/home/jovyan/work/Data/SRR4841864/fastqs_trimmed"
DIR_DAT_FILTER = "/home/jovyan/work/Data/SRR4841864/fastqs_filter"
DIR_CUR      = "/home/jovyan/work/GitRepo/Duke_CBB520_HW2"

THRES_SCORE = 25
THRES_LEN   = 50

In [2]:
os.listdir(DIR_DAT_TRIM)

['SRR4841864_1.trim.fastq', 'SRR4841864_2.trim.fastq']

## Streamming and Filtering Reads

In [7]:
%%time

### Generator
fn   = "SRR4841864_1.trim.fastq"      # file name
fp   = os.path.join(DIR_DAT_TRIM, fn) # file path
gen1 = stream_fastq(fp)               # generator

fn   = "SRR4841864_2.trim.fastq"      # file name
fp   = os.path.join(DIR_DAT_TRIM, fn) # file path
gen2 = stream_fastq(fp)               # generator

### Reset the file
fn      = "SRR4841864_1.filter.fastq"      # file name
out_fp1 = os.path.join(DIR_DAT_FILTER, fn) # file path
out1    = open(out_fp1, "w")

fn      = "SRR4841864_2.filter.fastq"      # file name
out_fp2 = os.path.join(DIR_DAT_FILTER, fn) # file path
out2    = open(out_fp2, "w")

### Stream, filter, and write
for cnt, (read1, read2) in enumerate(zip(gen1, gen2)):
    ### get list of phred scores
    phred1 = [ord(char)-33 for char in read1["qscore"]]
    phred2 = [ord(char)-33 for char in read2["qscore"]]
    
    ### high quality
    ishq1 = is_high_quality(phred1, threshold_score = THRES_SCORE, threshold_length = THRES_LEN)
    ishq2 = is_high_quality(phred2, threshold_score = THRES_SCORE, threshold_length = THRES_LEN)
    
    ### output read to file
    if ishq1 and ishq2:
        write_fastq(out_fp1, read1)
        write_fastq(out_fp2, read2)
        
out1.close()
out2.close()

CPU times: user 22min 37s, sys: 1min, total: 23min 38s
Wall time: 23min 39s


**Check the read1 and read2 still contain the same number of line**

In [9]:
%%bash
source config.sh
wc -l $FILTER/SRR4841864_1.filter.fastq
wc -l $FILTER/SRR4841864_2.filter.fastq

14136508 /home/jovyan/work/Data/SRR4841864/fastqs_filter/SRR4841864_1.filter.fastq
14136508 /home/jovyan/work/Data/SRR4841864/fastqs_filter/SRR4841864_2.filter.fastq


## Visualization of Raw, Trimmed, and Filtered Reads

In [28]:
fn       = "SRR4841864_1.fastq"              # file name
fp       = os.path.join(DIR_DAT_RAW, fn)     # file path
gen_raw1 = stream_fastq(fp)                  # generator

fn       = "SRR4841864_2.fastq"              # file name
fp       = os.path.join(DIR_DAT_RAW, fn)     # file path
gen_raw2 = stream_fastq(fp)                  # generator

fn        = "SRR4841864_1.trim.fastq"        # file name
fp        = os.path.join(DIR_DAT_TRIM, fn)   # file path
gen_trim1 = stream_fastq(fp)                 # generator

fn        = "SRR4841864_2.trim.fastq"        # file name
fp        = os.path.join(DIR_DAT_TRIM, fn)   # file path
gen_trim2 = stream_fastq(fp)                 # generator

fn        = "SRR4841864_1.filter.fastq"      # file name
fp        = os.path.join(DIR_DAT_FILTER, fn) # file path
gen_filt1 = stream_fastq(fp)                 # generator

fn        = "SRR4841864_2.filter.fastq"      # file name
fp        = os.path.join(DIR_DAT_FILTER, fn) # file path
gen_filt2 = stream_fastq(fp)                 # generator

########################################################
fig, axes = plt.subplots(ncols=2, nrows=3, figsize=(15, 9))
plt.subplots_adjust(hspace=0.4)

for cnt, reads in enumerate(zip(gen_raw1,  gen_raw2, 
                                gen_trim1, gen_trim2, 
                                gen_filt1, gen_filt2)):
    for ax, read in zip(axes.ravel(), reads):
        qscore  = read["qscore"]
        phred   = [ord(char)-33 for char in qscore]
        ax.plot(phred, alpha = 0.2)
        ax.axhline(y=THRES_SCORE, c="red")
    
    if cnt > 50:
        break

########################################################
### Set plot axes
ax1, ax2 = axes[0,:]
ax1.set_title("SRR4841864_1 (Raw)")
ax2.set_title("SRR4841864_2 (Raw)")

ax1, ax2 = axes[1,:]
ax1.set_title("SRR4841864_1 (Trimmed)")
ax2.set_title("SRR4841864_2 (Trimmed)")

ax1, ax2 = axes[2,:]
ax1.set_title("SRR4841864_1 (Filtered)")
ax2.set_title("SRR4841864_2 (Filtered)")

for ax in axes.ravel():
    ax.set_ylim([0, 42])
    ax.set_xlabel("Index of Bases")
    ax.set_ylabel("Phred Score")
pass